<a href="https://colab.research.google.com/github/JSJeong-me/TorchText/blob/main/petitions_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -r requirements.txt

  Using cached absl_py-0.11.0-py3-none-any.whl (127 kB)
  Using cached argon2_cffi-20.1.0-cp35-abi3-manylinux1_x86_64.whl (97 kB)
  Using cached async_generator-1.10-py3-none-any.whl (18 kB)
  Using cached attrs-20.3.0-py2.py3-none-any.whl (49 kB)
  Using cached beautifulsoup4-4.6.0-py3-none-any.whl (86 kB)
  Using cached bleach-3.2.1-py2.py3-none-any.whl (145 kB)
  Using cached cachetools-4.2.0-py3-none-any.whl (12 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached cffi-1.14.4-cp37-cp37m-manylinux1_x86_64.whl (402 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached colorama-0.4.4-py2.py3-none-any.whl (16 kB)
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
  Using cached Cython-0.29.14-cp37-cp37m-manylinux1_x86_64.whl (2.1 MB)
  Using cached defusedxml-0.6.0-py2.py3-none-any.whl (23 kB)
  Using cached fsspec-0.8.5-py3-none-any.whl (98 kB)
  Using cached future-0.18.2.tar.gz (829 kB)
  Using cached gensim-3.8.3-cp37-cp3

# 2.1 크롤링

[크롤링]

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup 
import time


result = pd.DataFrame()                                    

for i in range(594926, 595226):
    URL = "http://www1.president.go.kr/petitions/"+str(i)
 
    response = requests.get(URL)    
    html = response.text                                   
    soup = BeautifulSoup(html, 'html.parser')           

    title = soup.find('h3', class_='petitionsView_title')
    count = soup.find('span', class_='counter')           

    for content in soup.select('div.petitionsView_write > div.View_write'):
        content                                         

    a=[]
    for tag in soup.select('ul.petitionsView_info_list > li'): 
        a.append(tag.contents[1])

    if len(a) != 0:
        df1=pd.DataFrame({ 'start' : [a[1]],                
                           'end' : [a[2]],                     
                           'category' :  [a[0]],               
                           'count' : [count.text],             
                           'title': [title.text],              
                           'content': [content.text.strip()[0:13000]]                              
                         })

        result=pd.concat([result, df1])                        
        result.index = np.arange(len(result))             

    if i % 60 == 0:                                        
        print("Sleep 90seconds. Count:" + str(i)           
              +",  Local Time:"+ time.strftime('%Y-%m-%d', time.localtime(time.time()))
              +" "+ time.strftime('%X', time.localtime(time.time()))
              +",  Data Length:"+ str(len(result)))        
        time.sleep(10) 

Sleep 90seconds. Count:594960,  Local Time:2022-01-15 12:49:37,  Data Length:35
Sleep 90seconds. Count:595020,  Local Time:2022-01-15 12:49:58,  Data Length:94
Sleep 90seconds. Count:595080,  Local Time:2022-01-15 12:50:15,  Data Length:115
Sleep 90seconds. Count:595140,  Local Time:2022-01-15 12:50:30,  Data Length:117
Sleep 90seconds. Count:595200,  Local Time:2022-01-15 12:50:44,  Data Length:117


[크롤링 데이터 확인]

In [3]:
print(result.shape)

df = result
df.head()

(117, 6)


,start,end,category,count,title,content
0,2020-12-23,2021-01-22,보건복지,257,목포 *안과 드림렌즈사기,저는11월20일 목포*안과를 찾아 드림렌즈를 신청하였 습니다. 정상적인 접수와 검...
1,2020-12-23,2021-01-22,안전/환경,225,목포시 오염 실태를 아십니까?,지금 목포시 상황이 말이아닙니다.\r\n공무원들이 시민을 고소하고 환경오염으로 지적...
2,2020-12-23,2021-01-22,교통/건축/국토,237,탄원서,탄 원 서\n\r\n발신인 : 수익권증서 상의 1순위 질권자(채권자) 대표\r\n ...
3,2020-12-23,2021-01-22,안전/환경,180,확실한 코로나 예방을 위해 이중 마스크 착용을 강력하게 알려야 합니다.,코로나 확산 방지와 예방을 위한 가장 강력한 방법은 올바른 마스크 착용입니다.\r\...
4,2020-12-23,2021-01-22,육아/교육,373,긴급 돌봄과 긴급 보육의 구체화와 보육교사 대상 코로나 검사 실행 철회를 요구합니다.,안녕하세요. 저는 유치원에서 근무하고 있는 교사입니다. 비록 보육교사는 아니지만 현...


[데이터 엑셀로 저장]

In [4]:
!mkdir data

mkdir: cannot create directory ‘data’: File exists


In [5]:
df.to_csv('data/crawling.csv', index = False, encoding = 'utf-8-sig')

# 2.2 데이터 전처리

In [6]:
df.loc[1]['content']  # 전처리 전

'지금 목포시 상황이 말이아닙니다.\r\n공무원들이 시민을 고소하고 환경오염으로 지적한부분\r\n시정되지도 않고 시민들만 오폐수 문제로 골머리가 아플지경 입니다.\r\n저의 고향이지만 이렇게 오폐수로 망가져가는 모습을\r\n더는 방관할 수 없을것 같습니다.\r\n시의원과 목포시 공무원들 일은 하고있는지\r\n제데로 조사가 필요할 것 같습니다.\r\n제가 이렇게까지 고향을 생각할정도의 오폐수 문제가\r\n너무도 심각하다는것을 인지해주시고 정부에서 강력하게 잘못된 목포시 행정을 바로 잡아주세요.\r\n이게 무슨일입니까?\r\n난 보고도 도저히 이해가 되지않습니다.\n\r\n목포시 오염수 관련해서 바다를 오염 시키고 \r\n이런것을 고쳐낼 생각은 안하고 시민들을 고소 민사소송하겠다.\r\n협박하는게 현재 목포시 횡포입니다.\r\n밑에 관련정보를 조사해주신 분의 페이스북링크를\r\n남깁니다.\n\n [본 게시물의 일부 내용이 국민 청원 요건에 위배되어 관리자에 의해 수정되었습니다]'

[전처리]

In [7]:
import re

def remove_white_space(text):
    text = re.sub(r'[\t\r\n\f\v]', ' ', str(text))
    return text

def remove_special_char(text):
    text = re.sub('[^ ㄱ-ㅣ가-힣 0-9]+', ' ', str(text))
    return text

df.title = df.title.apply(remove_white_space)
df.title = df.title.apply(remove_special_char)

df.content = df.content.apply(remove_white_space)
df.content = df.content.apply(remove_special_char)

In [8]:
df.loc[1]['content']  # 전처리 후

'지금 목포시 상황이 말이아닙니다   공무원들이 시민을 고소하고 환경오염으로 지적한부분  시정되지도 않고 시민들만 오폐수 문제로 골머리가 아플지경 입니다   저의 고향이지만 이렇게 오폐수로 망가져가는 모습을  더는 방관할 수 없을것 같습니다   시의원과 목포시 공무원들 일은 하고있는지  제데로 조사가 필요할 것 같습니다   제가 이렇게까지 고향을 생각할정도의 오폐수 문제가  너무도 심각하다는것을 인지해주시고 정부에서 강력하게 잘못된 목포시 행정을 바로 잡아주세요   이게 무슨일입니까   난 보고도 도저히 이해가 되지않습니다    목포시 오염수 관련해서 바다를 오염 시키고   이런것을 고쳐낼 생각은 안하고 시민들을 고소 민사소송하겠다   협박하는게 현재 목포시 횡포입니다   밑에 관련정보를 조사해주신 분의 페이스북링크를  남깁니다     본 게시물의 일부 내용이 국민 청원 요건에 위배되어 관리자에 의해 수정되었습니다 '

# 2.3 토크나이징 및 변수 생성

[토크나이징]

In [9]:
!pip install konlpy 

In [10]:
from konlpy.tag import Okt

okt = Okt()

df['title_token'] = df.title.apply(okt.morphs)
df['content_token'] = df.content.apply(okt.nouns)

[파생변수 생성]

In [11]:
df['token_final'] = df.title_token + df.content_token

df['count'] = df['count'].replace({',' : ''}, regex = True).apply(lambda x : int(x))

print(df.dtypes)

df['label'] = df['count'].apply(lambda x: 'Yes' if x>=1000 else 'No')

start            object
end              object
category         object
count             int64
title            object
content          object
title_token      object
content_token    object
token_final      object
dtype: object


In [12]:
df_drop = df[['token_final', 'label']]

In [13]:
df_drop.head()

,token_final,label
0,"[목포, 안과, 드림, 렌즈, 사기, 저, 목포, 안과, 드림, 렌즈, 신청, 정상...",No
1,"[목포시, 오염, 실태, 를, 아십니까, 지금, 목포시, 상황, 말, 공무원, 시민...",No
2,"[탄원서, 원, 발신, 수익권, 증서, 상의, 순위, 질권, 채권자, 대표, 주, ...",No
3,"[확실한, 코로나, 예방, 을, 위해, 이중, 마스크, 착용, 을, 강력하게, 알려...",No
4,"[긴급, 돌봄과, 긴급, 보육, 의, 구체화, 와, 보육, 교사, 대상, 코로나, ...",No


[데이터 엑셀로 저장]

In [14]:
df_drop.to_csv('data/df_drop.csv', index = False, encoding = 'utf-8-sig')

# 2.4 단어 임베딩

[단어 임베딩]

In [15]:
from gensim.models import Word2Vec

embedding_model = Word2Vec(df_drop['token_final'], 
                           sg = 1, # skip-gram
                           size = 100, 
                           window = 2, 
                           min_count = 1, 
                           workers = 4
                           )

print(embedding_model)

model_result = embedding_model.wv.most_similar("음주운전")
print(model_result)

Word2Vec(vocab=5031, size=100, alpha=0.025)
[('장관', 0.9232786297798157), ('허용', 0.9210355281829834), ('정보', 0.9209083318710327), ('언제', 0.9191786050796509), ('충격', 0.9189960956573486), ('하라', 0.9180748462677002), ('대기업', 0.9178268909454346), ('매니저', 0.9171040654182434), ('죄', 0.9170569181442261), ('긴급', 0.9170064926147461)]


[임베딩 모델 저장 및 로드]

In [16]:
from gensim.models import KeyedVectors

embedding_model.wv.save_word2vec_format('data/petitions_tokens_w2v') # 모델 저장
loaded_model = KeyedVectors.load_word2vec_format('data/petitions_tokens_w2v') # 모델 로드

model_result = loaded_model.most_similar("음주운전")
print(model_result)

[('장관', 0.9232786297798157), ('허용', 0.9210355281829834), ('정보', 0.9209083318710327), ('언제', 0.9191786050796509), ('충격', 0.9189960956573486), ('하라', 0.9180748462677002), ('대기업', 0.9178268909454346), ('매니저', 0.9171040654182434), ('죄', 0.9170569181442261), ('긴급', 0.9170064926147461)]


# 2.5 실험 설계

[데이터셋 분할 및 저장]

In [17]:
from numpy.random import RandomState

rng = RandomState()

tr = df_drop.sample(frac=0.8, random_state=rng)
val = df_drop.loc[~df_drop.index.isin(tr.index)]

tr.to_csv('data/train.csv', index=False, encoding='utf-8-sig')
val.to_csv('data/validation.csv', index=False, encoding='utf-8-sig')

[Field클래스 정의]

In [18]:
!!pip install -U torchtext==0.8.0

['Collecting torchtext==0.8.0',
 '  Using cached torchtext-0.8.0-cp37-cp37m-manylinux1_x86_64.whl (6.9 MB)',
 'Requirement already satisfied: numpy in /usr/local/lib/python3.7/dist-packages (from torchtext==0.8.0) (1.19.5)',
 'Requirement already satisfied: requests in /usr/local/lib/python3.7/dist-packages (from torchtext==0.8.0) (2.23.0)',
 'Requirement already satisfied: tqdm in /usr/local/lib/python3.7/dist-packages (from torchtext==0.8.0) (4.62.3)',
 'Requirement already satisfied: torch in /usr/local/lib/python3.7/dist-packages (from torchtext==0.8.0) (1.10.1)',
 'Requirement already satisfied: certifi>=2017.4.17 in /usr/local/lib/python3.7/dist-packages (from requests->torchtext==0.8.0) (2021.10.8)',
 'Requirement already satisfied: chardet<4,>=3.0.2 in /usr/local/lib/python3.7/dist-packages (from requests->torchtext==0.8.0) (3.0.4)',
 'Requirement already satisfied: idna<3,>=2.5 in /usr/local/lib/python3.7/dist-packages (from requests->torchtext==0.8.0) (2.10)',
 'Requirement a

In [19]:
#!pip install torch==1.6 torchtext==0.7

In [20]:
!pip install https://github.com/pytorch/text/archive/master.zip

  Using cached https://github.com/pytorch/text/archive/master.zip
  ERROR: Failed building wheel for torchtext
  Running setup.py clean for torchtext
Failed to build torchtext
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.8.0
    Uninstalling torchtext-0.8.0:
      Successfully uninstalled torchtext-0.8.0
    Running setup.py install for torchtext ... error
  Rolling back uninstall of torchtext
  Moving to /usr/local/lib/python3.7/dist-packages/torchtext-0.8.0.dist-info/
   from /usr/local/lib/python3.7/dist-packages/~orchtext-0.8.0.dist-info
  Moving to /usr/local/lib/python3.7/dist-packages/torchtext/
   from /usr/local/lib/python3.7/dist-packages/~orchtext
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-req-build-_5334c4n/setup.py'"'"'; __file__='"'"'/tmp/pip-req-build-_5334c4n/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exi

In [23]:
!pip install -U torchtext

  Using cached torchtext-0.11.1-cp37-cp37m-manylinux1_x86_64.whl (8.0 MB)
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.8.0
    Uninstalling torchtext-0.8.0:
      Successfully uninstalled torchtext-0.8.0


In [24]:
import torchtext
from torchtext.data import Field

def tokenizer(text):
    text = re.sub('[\[\]\']', '', str(text))
    text = text.split(', ')
    return text

TEXT = Field(tokenize=tokenizer)
LABEL = Field(sequential = False)

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.


[데이터 불러오기]

In [25]:
from torchtext.data import TabularDataset

train, validation = TabularDataset.splits(
    path = 'data/',
    train = 'train.csv',
    validation = 'validation.csv',
    format = 'csv',
    fields = [('text', TEXT), ('label', LABEL)],
    skip_header = True
)

print("Train:", train[0].text,  train[0].label)
print("Validation:", validation[0].text, validation[0].label)

Train: ['피싱', '사기', '를', '예방', '하기', '위해', '중국', '공안', '과', '공조', '하여', '함정수사', '를', '진행', '부탁', '합니다', '피싱', '사기', '예방', '위해', '중국', '공안', '공조', '함정수사', '진행', '부탁', '보이스피싱', '메신저', '피싱', '대부분', '해외', '진행', '중국', '공안', '공조', '함정수사', '진행', '필요'] No
Validation: ['확실한', '코로나', '예방', '을', '위해', '이중', '마스크', '착용', '을', '강력하게', '알려야', '합니다', '코로나', '확산', '방지', '예방', '위', '가장', '방법', '마스크', '착용', '상식', '그', '예방', '효과', '자랑', '마스크', '장만', '착용', '것', '이중', '두', '장', '겹', '착용', '본인', '더', '예방', '효과', '볼', '것', '타인', '볼', '때', '더', '수', '저', '직장', '동료', '뜻', '지난', '주말', '간', '이중', '마스크', '착용', '결과', '생각', '호흡', '더', '느낌', '오히려', '습기', '덜', '차고', '착용', '이중', '마스크', '다시', '과정', '장점', '마스크', '장만', '것', '두', '장', '겹', '것', '단', '예방', '효과', '지금', '당장', '온', '국민', '실천', '널리', '마스크', '두', '장', '겹', '점', '수', '그', '부분', '감안', '예방', '효과', '초점', '즉시', '실천', '생각', '지금', '코로나', '대한', '가장', '백신', '마스크', '착용', '그', '백신', '더', '이중', '접종', '국민', '모두', '불안감', '일상생활', '쇼', '지금', '정부', '국민', '권고', '국민', '모두', '이중',

/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.


[단어장 및 DataLoader 정의]

In [26]:
import torch
from torchtext.vocab import Vectors
from torchtext.legacy.data import BucketIterator

vectors = Vectors(name="data/petitions_tokens_w2v")

TEXT.build_vocab(train, vectors = vectors, min_freq = 1, max_size = None)
LABEL.build_vocab(train)

vocab = TEXT.vocab

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iter, validation_iter = BucketIterator.splits(
    datasets = (train, validation),
    batch_size = 8,
    device = device,
    sort = False
)

print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT.vocab.vectors.shape))

임베딩 벡터의 개수와 차원 : torch.Size([4412, 100]) 


# 2.6 TextCNN

[TextCNN 모델링]

In [27]:
import torch.nn as nn   
import torch.optim as optim 
import torch.nn.functional as F 

class TextCNN(nn.Module): 
    
    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins, num_class):
        
        super(TextCNN, self).__init__()
        
        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        self.embed.weight.data.copy_(vocab_built.vectors)      
    
        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        self.relu = nn.ReLU()                
        self.dropout = nn.Dropout(0.4)         
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, num_class)     
        
    def forward(self, x):  
      
        emb_x = self.embed(x)           
        emb_x = emb_x.unsqueeze(1)  

        con_x = [self.relu(conv(emb_x)) for conv in self.convs]       

        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]    
        
        fc_x = torch.cat(pool_x, dim=1) 
        fc_x = fc_x.squeeze(-1)       
        fc_x = self.dropout(fc_x)         

        logit = self.fc(fc_x)     
        
        return logit

[모델 학습 함수 정의]

In [28]:
def train(model, device, train_itr, optimizer):
    
    model.train()                               
    corrects, train_loss = 0.0,0        
    
    for batch in train_itr:
        
        text, target = batch.text, batch.label      
        text = torch.transpose(text, 0, 1)          
        target.data.sub_(1)                                 
        text, target = text.to(device), target.to(device)  

        optimizer.zero_grad()                           
        logit = model(text)                         
    
        loss = F.cross_entropy(logit, target)   
        loss.backward()  
        optimizer.step()  
        
        train_loss += loss.item()    
        result = torch.max(logit,1)[1] 
        corrects += (result.view(target.size()).data == target.data).sum()
        
    train_loss /= len(train_itr.dataset)
    accuracy = 100.0 * corrects / len(train_itr.dataset)

    return train_loss, accuracy

[모델 평가 함수 정의]

In [29]:
def evaluate(model, device, itr):
    
    model.eval()
    corrects, test_loss = 0.0, 0

    for batch in itr:
        
        text = batch.text
        target = batch.label
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        logit = model(text)
        loss = F.cross_entropy(logit, target)

        test_loss += loss.item()
        result = torch.max(logit,1)[1]
        corrects += (result.view(target.size()).data == target.data).sum()

    test_loss /= len(itr.dataset) 
    accuracy = 100.0 * corrects / len(itr.dataset)
    
    return test_loss, accuracy

[모델 학습 및 성능 확인]

In [30]:
model = TextCNN(vocab, 100, 10, [3, 4, 5], 2).to(device)
print(model)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

optimizer = optim.Adam(model.parameters(), lr=0.001)

best_test_acc = -1

for epoch in range(1, 3+1):
 
    tr_loss, tr_acc = train(model, device, train_iter, optimizer) 
    print('Train Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, tr_loss, tr_acc))
    
    val_loss, val_acc = evaluate(model, device, validation_iter)
    print('Valid Epoch: {} \t Loss: {} \t Accuracy: {}%'.format(epoch, val_loss, val_acc))
        
    if val_acc > best_test_acc:
        best_test_acc = val_acc
        
        print("model saves at {} accuracy".format(best_test_acc))
        torch.save(model.state_dict(), "TextCNN_Best_Validation")
    
    print('-----------------------------------------------------------------------------')

TextCNN(
  (embed): Embedding(4412, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 10, kernel_size=(3, 100), stride=(1, 1))
    (1): Conv2d(1, 10, kernel_size=(4, 100), stride=(1, 1))
    (2): Conv2d(1, 10, kernel_size=(5, 100), stride=(1, 1))
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=30, out_features=2, bias=True)
)
Train Epoch: 1 	 Loss: 0.08803768487686807 	 Accuracy: 53.19148635864258%
Valid Epoch: 1 	 Loss: 0.09471270053283028 	 Accuracy: 39.130435943603516%
model saves at 39.130435943603516 accuracy
-----------------------------------------------------------------------------
Train Epoch: 2 	 Loss: 0.08812317886251084 	 Accuracy: 50.0%
Valid Epoch: 2 	 Loss: 0.09251731634140015 	 Accuracy: 39.130435943603516%
-----------------------------------------------------------------------------
Train Epoch: 3 	 Loss: 0.08618838229077927 	 Accuracy: 58.51063537597656%
Valid Epoch: 3 	 Loss: 0.09059126221615335 	 Accuracy: 39.130435943603516%